In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import keras
import keras.backend as k

In [5]:
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [9]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1

/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1


In [10]:
!pwd

/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1


In [47]:
PATH="/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/"

In [48]:
train_imgs_path=pd.read_csv(PATH+'Stanford-MURA-master/train_image_paths.csv')
train_labels=pd.read_csv(PATH+'Stanford-MURA-master/train_labeled_studies.csv')
test_imgs_path=pd.read_csv(PATH+'Stanford-MURA-master/valid_image_paths.csv')
test_labels=pd.read_csv(PATH+'Stanford-MURA-master/valid_labeled_studies.csv')


In [49]:
train_labels['Body_Part']=train_labels['Img_Path'].apply(lambda x: str(x.split('/')[2])[3:])
train_labels['Study_Type']=train_labels['Img_Path'].apply(lambda x: str(x.split('/')[4])[:6])
test_labels['Body_Part']=test_labels['Img_Path'].apply(lambda x: str(x.split('/')[2])[3:])
test_labels['Study_Type']=test_labels['Img_Path'].apply(lambda x: str(x.split('/')[4])[:6])

In [50]:
def read_image(Path):
    img=cv2.imread(Path)
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(224,224))
    #print (img.shape)
    img=np.array(img)
    #img=np.resize(img,(224,224))
    #print (img.shape)
    img=img/255.
    return img

In [51]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions

In [52]:
IMG_SIZE=(224,224)

In [53]:
train_datagen=ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

In [54]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [56]:
train_imgs=list(train_imgs_path.Img_Path.values)

In [57]:
valid_imgs=list(test_imgs_path.Img_Path.values)

In [58]:
train_generator = train_datagen.flow_from_directory(
        os.path.join(PATH,'MURA-v1.1/train'),  # this is the target directory
        target_size=IMG_SIZE,  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='binary') 

Found 36812 images belonging to 7 classes.


In [59]:
validation_generator = test_datagen.flow_from_directory(
        os.path.join(PATH,'MURA-v1.1/valid'),
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='binary')

Found 3197 images belonging to 7 classes.


In [60]:
model=VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [61]:
model.output_shape

(None, 7, 7, 512)

In [62]:
from keras.layers import  Convolution2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.models import  Sequential
from tensorflow.keras.optimizers import Adam

In [63]:
model1=Sequential()
model1.add(model)
model1.add(Flatten(input_shape=model.output_shape[1:]))
model1.add(Dropout(0.5))
#model1.add(Dense(128, activation='relu'))
#model1.add(Dropout(0.5))
model1.add(Dense(1, activation='softmax'))

In [64]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 14,739,777
Non-trainable params: 0
_________________________________________________________________


In [65]:
for layer in model.layers:
    layer.trainable=False

In [66]:
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])

In [71]:
batch_size=128
import os

In [67]:
model1.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

62/62 [==============================] - 1472s 24s/step - loss: -1616.6049 - accuracy: 0.1290 - val_loss: -317.0935 - val_accuracy: 0.1138


In [72]:
model1.fit_generator(
        train_generator,
        steps_per_epoch=640 // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=640 // batch_size)

5/5 [==============================] - 184s 39s/step - loss: -3666.8972 - accuracy: 0.1250 - val_loss: -284.2094 - val_accuracy: 0.1437


In [73]:
 
 #Saving the model weights
model.save("bonemodel1.h5")
print("Saved model to disk")

Saved model to disk
